# Bakery Sales

### Base de datos de ventas de panadería

#### Me gustaría saber ¿Los clientes más cercanos(en localidad) son quienes tienen mayor consumo?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## * Importación de librerías

In [ ]:
# Importamos las librerías a utilizar
import numpy as np
import pandas as pd
# Librerías para grafícos
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
%matplotlib inline
import datetime as dt

# Lectura de Base de datos

In [ ]:
# Importamos base de datos 
DB_Sales = pd.read_csv('../input/bakery-sales/Bakery Sales.csv',parse_dates = ['datetime'])
DB_Sales

# Limpieza Base de datos

In [ ]:
DB_Sales.info()

In [ ]:
# Eliminaremos los renglones que tengan NAN values en su sección de datetime
DB_Sales.dropna(subset =['datetime'], inplace = True)
DB_Sales

In [ ]:
# Cambiaremos los valores NAN por 0
DB_Sales.replace(np.nan, 0, inplace = True)
DB_Sales.tail()

In [ ]:
DB_Sales['place'].unique()

In [ ]:
DB_Sales['day of week'].unique()

In [ ]:
DB_Sales.describe()

In [ ]:
# Cambiaremos los lugares en coreano a sus nombres legibles en ingles
DB_Sales['place'].replace({'소양동' : 'Soyang-dong', '효자 3동' : 'Hyoja3-dong', '후평 1동' : 'Hoopyeong1-dong', '후평 2동' : 'Hoopyeong2-dong', 
                           '석사동' : 'Seoksa-dong', '퇴계동' : 'Toegye-dong', '동면' : 'Dongmyeon', '후평 3동': 'Hoopyeong3-dong', "신사우동" : 'Sinsawoo-dong',
                          "강남동" : 'Gangnam-dong', "효자 1동": 'Hyoja1-dong', '조운동' : 'Jowoon-dong', '교동' : 'Gyo-dong', '효자 2동' : 'Hyoja2-dong', '약사명동' : 'Yaksamyeong-dong',
                          '근화동': 'Geunhwa-dong', '동내면' : 'Dongnae-myeon', '신동면' : 'Sindong-myeon', '교동 ': 'Gyo-dong'}, inplace = True)

# EDA

In [ ]:
# Conoceremos que lugar es el que mayor ingreso nos genera
#DB_place_sales = DB_Sales.groupby(DB_Sales['place'], as_index = False)['total'].sum().sort_values(by='total', ascending=False)
DB_place_sales = DB_Sales.groupby(DB_Sales['place'])['total'].sum().sort_values(ascending=False)
DB_place_sales = pd.DataFrame(DB_place_sales)
DB_place_sales

In [ ]:
# Valores de que lugar y día son los más beneficiosos
#DB_place_day = DB_Sales.groupby(['place','day of week']).agg({'total':sum})
DB_place_day = DB_Sales.groupby(['place','day of week'], as_index = False).agg({'total':sum})
#DB_place_day = pd.DataFrame(DB_place_day)
DB_place_day.head(25)

In [ ]:
DB_place_day.shape

In [ ]:
# Generaremos una scatter circle plot
fig = go.Figure(data=[go.Scatter(
    x=DB_place_sales.index[::],
    y= DB_place_sales['total'][::],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(DB_place_sales['total'][::]/60000),
        showscale=True
        )
)])

fig.update_layout(
    title='Ganancias totales por estado',
    xaxis_title="Countries",
    yaxis_title="Ganancias Totales",
    template='plotly_white'
)
fig.show()

In [ ]:
# Utilizaremos un pie chart para vizualizar de manera individual los días y sus ingresos de la mejor tienda
fig = px.pie(DB_place_day, values=DB_place_day['total'][8:13], names=DB_place_day['day of week'][8:13],
             title='Dongmyeon',
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

In [ ]:
# Visualización completa de todas nuestras sucursales y sus ingresos obtenidos por día de la semana durante 10 meses
fig = px.sunburst(DB_place_day,path=['place','day of week','total'],
                 values='total',color='place',)
fig.update_layout(height=1000,title_text='Análisis de productos')
fig.show()

In [ ]:
# Agregaremos 4 columnas más para tener año, mes ,día y hora por separado.
DB_Sales['dates'] = DB_Sales['datetime'].dt.strftime("%Y-%m")
DB_Sales['month'] = DB_Sales['datetime'].dt.month
DB_Sales['day'] = DB_Sales['datetime'].dt.day
DB_Sales['hour'] = DB_Sales['datetime'].dt.hour
DB_Sales.head()

In [ ]:
# Agrupación de las gananciás acumuladas por meses
DB_month_sale = DB_Sales.groupby(DB_Sales['dates'], as_index = False)['total'].sum()
DB_month_sale

In [ ]:
# Vizualización grafíca de ganancias por meses
trace1 = go.Scatter(x = DB_month_sale['dates'],
              y = DB_month_sale['total'],
              name = 'Meses',
              mode='lines')


## ACTIVIDAD incluye aquí los países que analizaste en la actividad anterior

layout = go.Layout(title="Ganancias Mensuales",
                  xaxis = dict(title='Meses',zeroline= False,
                         gridcolor='rgb(183,183,183)',showline=True),
                  yaxis = dict(title='Ganancias',zeroline= False,
                            gridcolor='rgb(183,183,183)',showline=True),
                  font = dict(family='Courier New, monospace', size=12, color='rgb(0,0,0)'),
                  legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'))
    
data = [trace1] # trace3, trace4, trace5, trace6]
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
# Agrupación del total de productos consumidos por mes
DB_month_products = DB_Sales.groupby(DB_Sales['dates'], as_index = False)['angbutter','croissant','tiramisu croissant',
                                                                          'plain bread'].sum()
DB_month_products = DB_month_products.drop(11)
DB_month_products

In [ ]:
# Vizualización para observar si hubo comportamientos en la cantidad de productos vendidos
x_data = ['angbutter', 'croissant',
          'tiramisu croissant', 'plain bread']


y_data = [DB_month_products['angbutter'], DB_month_products['croissant'], DB_month_products['tiramisu croissant'],
         DB_month_products['plain bread']]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)']

fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1)
        )

fig.update_layout(
    title='Ventas mensuales de los 4 productos más vendidos',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()

# Conclusiones

## 1- Confirmamos que Dongmyeon que es la sucursal sede es la principal fuente de ingresos

## 2- Se puede observar que los días con respecto a todas las sucursales que mejores ventas tienen son Domingo y Jueves

## 3- De julio a Abril las ventas no generaron gananciás menores a los 4 M

## 4- En la grafíca de bigote que las ventas más bajas son en el ultimo mes registrado; probablemente la información no este completa.